# Aytekin Erdogan 2532703 Lane Detection Pipeline

### All necessary explanation is commented inside the code.


In [1]:
# Import necessary libraries
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt


### First calibrate the camera and remove distortion effects

In [2]:
def camera_calibration(img_path,cal_img_list):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = cal_img_list

    # Step through the list and search for chessboard corners
    for fname in images:
#         print(img_path + fname)
        img = cv2.imread(img_path + fname)
        #plt.imshow(img)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        if ret == True:
            # add object points, image points
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objectPoints=objpoints,
                                   imagePoints=imgpoints,
                                   imageSize=gray.shape[::-1],
                                   cameraMatrix=None,
                                   distCoeffs=None)
    return mtx, dist

In [3]:
def undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [4]:
def unwarp(img):
    src = np.float32([[200,img.shape[0]], [1130,img.shape[0]],[590,450],[685,450]])
    # For destination points, arbitrarily choose some points that is nice fit for displaying our warped result 
    dst = np.float32([[300,img.shape[0]], [1070, img.shape[0]], [300,0], [1070, 0]])
    
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size)

    # Return the output image and corresponding matrix
    return warped, M

In [5]:
def sobel_combined(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # Convert to hls color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    ret,s_binary = cv2.threshold(s_channel.astype('uint8'), 88, 190, cv2.THRESH_BINARY)
    
    sobel_kernel = 9
    # Pass images through sobel filter for both grayscale and s channel
    abs_sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel))
    abs_sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))

    # Scale to 8-bit (0 - 255) and type = np.uint8
    scaled_x = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    scaled_y = np.uint8(255*abs_sobely/np.max(abs_sobely))
    
    # Create a binary mask
    # Combine sobel outputs of S channel and grayscale channelwise
    binary_outputx = np.zeros_like(scaled_x)
    binary_outputx[((scaled_x >= 25)&(scaled_x <= 50))] = 1
    
    binary_outputy = np.zeros_like(scaled_y)
    binary_outputy[((scaled_y >= 50)&(scaled_y <= 150))] = 1
    
    # Combine binary images for x and y channels
    # Sobel abs thresholded img
    comb_xy = cv2.bitwise_or(np.uint8(binary_outputy),np.uint8(binary_outputx))
    
    # Calculate the magnitude of sobel images seperately for grayscale
    abs_xy = np.sqrt(abs_sobelx**2 + abs_sobely**2)
    
    scaled = np.uint8(255*abs_xy/np.max(abs_xy))

    # Create a binary mask where mag thresholds are met
    binary_output_mag = np.zeros_like(abs_xy)
    binary_output_mag[(scaled <= 250) & (scaled >= 50)] = 1
    
    # Calculate the direction of the gradient 
    g_dir = np.arctan2(abs_sobelx, abs_sobely)
    
    # Create a binary mask where direction thresholds are met
    binary_output_dir = np.zeros_like(g_dir)
    binary_output_dir[((g_dir >= 0.7) & (g_dir <= 1.3))] = 1
    
    kernel_1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    kernel_2 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel_3 = cv2.getStructuringElement(cv2.MORPH_CROSS,(7,7))
    
    combined = np.zeros_like(binary_output_dir)
    combined[(comb_xy == 1) | ((binary_output_mag == 1) & (binary_output_dir == 1)) | (s_binary >=150)] = 1
    
    kernel = np.ones((3,3), np.uint8)
    combined = cv2.morphologyEx(combined.astype(np.uint8), cv2.MORPH_OPEN, kernel)
    
    return combined

In [6]:
def sobel_combined_new(img):
    hsv_min = np.array([0, 70, 70])
    hsv_max = np.array([50, 255, 255])
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    yellow = cv2.inRange(hsv,hsv_min,hsv_max )
    yellow = np.uint8(255*yellow/np.max(yellow))
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist = cv2.equalizeHist(gray)
    _, white = cv2.threshold(hist, thresh=250, maxval=255, type=cv2.THRESH_BINARY)
    white = np.uint8(255*white/np.max(white))
    
    combined = np.zeros_like(gray)
    combined[(white >= 250) | (yellow >= 250)] = 1
    return combined

In [7]:
# List test images
test_images = os.listdir("inputs")
#print(test_images)

#Set output images directory
output_dir = "outputs/"

In [8]:
def sobel_combined_old(img):
    hsv_min = np.array([0, 70, 70])
    hsv_max = np.array([50, 255, 255])
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
#     print("x")
#     print(hsv.shape)
    min_th_ok = np.all(hsv > hsv_min)
#     print("x")
    max_th_ok = np.all(hsv < hsv_max, axis=2)
#     print("x")
    yellow = np.logical_and(min_th_ok, max_th_ok)
#     print("x")
    height, width = img.shape[:2]
    binary = np.zeros(shape=(height, width), dtype=np.uint8)
#     print("x")
    binary = np.logical_or(binary, yellow)
#     print("x")
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     print("x")
    hist = cv2.equalizeHist(gray)
    _, white = cv2.threshold(hist, thresh=250, maxval=255, type=cv2.THRESH_BINARY)
#     print("x")
    binary = np.logical_or(binary, white)
#     print("x")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     print("x")
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=9)
#     print("x")
    sobel_mag = np.sqrt(sobel_x ** 2 + sobel_y ** 2)
    sobel_mag = np.uint8(sobel_mag / np.max(sobel_mag) * 255)
#     print("x")
    _, sobel_mag = cv2.threshold(sobel_mag, 50, 1, cv2.THRESH_BINARY)
#     print("x")
    binary = np.logical_or(binary, sobel_mag.astype(bool))
#     print("x")
    return binary
    

In [9]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255, sthresh_min=0, sthresh_max=255):
    #Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Take the derivative in x or y given orient = 'x' or 'y'
    # Take the absolute value of the derivative or gradient
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
        abs_ssobel = np.absolute(cv2.Sobel(s_channel, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
        abs_ssobel = np.absolute(cv2.Sobel(s_channel, cv2.CV_64F, 0, 1))
    
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled = np.uint8(255*abs_sobel/np.max(abs_sobel))
    s_scaled = np.uint8(255*abs_ssobel/np.max(abs_ssobel))
    
    # Create a mask of 1's where the scaled gradient magnitude
    binary_output = np.zeros_like(scaled)
    binary_output[((scaled >= thresh_min)&(scaled <= thresh_max))] = 1
    sbinary_output = np.zeros_like(s_scaled)
    sbinary_output = ((s_scaled >= sthresh_min)&(s_scaled <= sthresh_max))
    img_bwo = cv2.bitwise_or(np.uint8(sbinary_output),np.uint8(binary_output))
    
    return img_bwo
    

In [10]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    
    # Take the gradient in x and y separately
    x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    xs = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    ys = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Calculate the magnitude 
    abs_xy = np.sqrt(x**2 + y**2)
    abs_xys = np.sqrt(xs**2 + ys**2)
    
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled = np.uint8(255*abs_xy/np.max(abs_xy))
    s_scaled = np.uint8(255*abs_xys/np.max(abs_xys))
    
    # Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(abs_xy)
    binary_output[(scaled <= mag_thresh[1]) & (scaled >= mag_thresh[0])|(s_scaled <= mag_thresh[1]) & (s_scaled >= mag_thresh[0])] = 1
    
    # Return this mask as your binary_output image
    return binary_output

In [11]:
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    # Take the gradient in x and y separately
    x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Calculate the magnitude 
    abs_x = np.absolute(x)
    abs_y = np.absolute(y)
    
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    g_dir = np.arctan2(abs_y, abs_x)
    
    # Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(g_dir)
    binary_output[(g_dir >= thresh[0]) & (g_dir <= thresh[1])] = 1
    
    # Return this mask as your binary_output image
    return binary_output

In [12]:
def find_lane_pixels(binary_warped):
    # Take the histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Find the peak of the left and right halves of the histogram to find the starting point for the left and right lines
    midpoint = int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # H-parameters
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = int(binary_warped.shape[0]//nwindows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        
        # Finding the four below boundaries of the window
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current -margin
        win_xright_high = rightx_current +margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find lane pixels
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    # Visualize colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

   
    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img, ploty, left_fitx, right_fitx, left_fit, right_fit

In [13]:
def curvature_radius (leftx, rightx, img_shape, xm_per_pix=3.7/800, ym_per_pix = 25/720):
    ploty = np.linspace(0, img_shape[0] - 1, img_shape[0])
    
    leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y
    
    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(ploty, leftx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(ploty, rightx, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 25/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/800 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    y_eval = np.max(ploty)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # radius of curvature is in meters
    return (left_curverad, right_curverad)

In [14]:

def show_output(undist, warped, Minv, ploty, left_fitx, right_fitx, left_fit, right_fit):
    # Calculate radius of curvature
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curvature =  ((1 + (2*left_fit_cr[0] *y_eval*ym_per_pix + left_fit_cr[1])**2) **1.5) / np.absolute(2*left_fit_cr[0])
    right_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
#     print((right_curvature+left_curvature)/2)
    
    # Calculate vehicle center
    #left_lane and right lane bottom in pixels
    left_lane_bottom = (left_fit[0]*y_eval)**2 + left_fit[0]*y_eval + left_fit[2]
    right_lane_bottom = (right_fit[0]*y_eval)**2 + right_fit[0]*y_eval + right_fit[2]
    avg_curvature = (right_curvature + left_curvature)/2
#     print(avg_curvature)
    
    # Lane center as mid of left and right lane bottom                        
    lane_center = (left_lane_bottom + right_lane_bottom)/2.
    center_image = 640
    center = (lane_center - center_image)*xm_per_pix #Convert to meters
#     print(center)
    
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    cmon = np.flipud(np.transpose(np.vstack([right_fitx, ploty]))).copy()
    pts_right = np.array([cmon])
    pts = np.hstack((pts_left, pts_right))


    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
#     print("points: ")
#     print(pts)
#     print("pts_vals: ")
#     print(pts.shape[1])
#     print(pts[0][pts.shape[1]-1][0])
#     print(pts[0,0])
#     print("pts_len: ")
#     print(pts[0,-1])
#     print(pts[0,300])
#     print("check")
    Minv = np.linalg.inv(Minv)
    pts_prev = pts

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    curv_info = "Curvuture: " + str(avg_curvature) +"m"
    if center < 0:
        center_info = "Vehicle is " + str(np.abs(center)) + "m left of center"
    else:
        center_info = "Vehicle is " + str(np.abs(center)) + "m right of center"
    cv2.putText(result,center_info,(30,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
    cv2.putText(result,curv_info,(30,30), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
    plt.imshow(result)
    
    return result


## Test Pipeline

In [15]:
# Change current working directory
os.chdir("/home/aerdog14/REPO/courses/metu/mmi702-project/part1- pipeline")
# List camera calibration images
calibration_imgs = os.listdir("camera_cal/")

# Get camera and distortion matrices
mtx, dst = camera_calibration("camera_cal/", calibration_imgs)

# Test with chessboard if undistortion works well or not 
test_img = cv2.imread("camera_cal/calibration1.jpg")
im= undistort(test_img,mtx,dst)
# plt.imshow(im)

#Set output images directory
output_dir = "outputs/"
cv2.imwrite(output_dir + "original_calibration1.jpg" , test_img)
cv2.imwrite(output_dir + "undis_calibration1.jpg", im)

True

In [16]:
# List test images
# test_images = os.listdir("rev_inputs")
test_images = os.listdir("test_images")
# print(test_images)

#Set output images directory
# output_dir = "rev_outputs/"
output_dir = "outputs/"

In [17]:
for test_im in test_images:
    if test_im.endswith(".jpeg") or test_im.endswith(".jpg"):
        img = image = mpimg.imread("test_images/" + test_im)
        undist= undistort(img,mtx,dst)
        cv2.imwrite(output_dir + "undist_" +test_im, undist)
#         plt.imshow(undist)
        
        combined = sobel_combined(undist)
        combined = combined *255
#         plt.imshow(combined, cmap="gray")
        cv2.imwrite(output_dir + "binary_combo_" +test_im, combined)

        im, M = unwarp(combined)
        cv2.imwrite(output_dir + "warped_" +test_im, im)
#         plt.imshow(im)

        out_img, ploty, left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(im)
        cv2.imwrite(output_dir + "fit_" +test_im, out_img)
#         plt.imshow(out_img)


        result = show_output(undist, im, M, ploty, left_fitx, right_fitx, left_fit, right_fit)
        cv2.imwrite(output_dir + "output_" + test_im, result)


In [18]:
warp_zero = np.zeros_like(im).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
cmon = np.flipud(np.transpose(np.vstack([right_fitx, ploty]))).copy()
pts_right = np.array([cmon])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

M = np.linalg.inv(M)
# Warp the blank back to original image space using inverse perspective matrix (Minv)
newwarp = cv2.warpPerspective(color_warp, M, (image.shape[1], image.shape[0])) 
# Combine the result with the original image
result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
plt.imshow(result)

In [19]:
prev_pts = []
latent1_pts = []
latent2_pts = []
latent3_pts = []
isFirstFrame = True;
isSecondFrame = True;
isThirdFrame = True;

def show_output_new(undist, warped, Minv, ploty, left_fitx, right_fitx, left_fit, right_fit):
    global isFirstFrame,isSecondFrame, isThirdFrame
    global prev_pts, latent2_pts, latent1_pts, latent3_pts
    # Calculate radius of curvature
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curvature =  ((1 + (2*left_fit_cr[0] *y_eval*ym_per_pix + left_fit_cr[1])**2) **1.5) / np.absolute(2*left_fit_cr[0])
    right_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
#     print((right_curvature+left_curvature)/2)
    
    # Calculate vehicle center, left_lane and right lane bottom in pixels
    left_lane_bottom = (left_fit[0]*y_eval)**2 + left_fit[0]*y_eval + left_fit[2]
    right_lane_bottom = (right_fit[0]*y_eval)**2 + right_fit[0]*y_eval + right_fit[2]
    avg_curvature = (right_curvature + left_curvature)/2
#     print(avg_curvature)
    
    # Lane center as mid of left and right lane bottom                        
    lane_center = (left_lane_bottom + right_lane_bottom)/2.
    center_image = 640
    center = (lane_center - center_image)*xm_per_pix #Convert to meters
#     print(center)
    
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    cmon = np.flipud(np.transpose(np.vstack([right_fitx, ploty]))).copy()
    pts_right = np.array([cmon])
    pts = np.hstack((pts_left, pts_right))
    
    if isThirdFrame and not(isSecondFrame) and not(isFirstFrame):
        latent3_pts = latent2_pts
        latent2_pts = latent1_pts
        latent1_pts = pts
        prev_pts = pts
        isThirdFrame = False
    elif isThirdFrame and isSecondFrame and not(isFirstFrame):
        latent2_pts = latent1_pts
        latent1_pts = pts
        prev_pts = pts
        isSecondFrame = False
    elif isFirstFrame:
        isFirstFrame = False
        prev_pts = pts
        latent1_pts = pts
    else:
#         print( "previous")
#         print(prev_pts[0][prev_pts.shape[1]-1][0])
#         print(pts[0][pts.shape[1]-1][0])
        if abs(prev_pts[0][prev_pts.shape[1]-1][0]-pts[0][pts.shape[1]-1][0]) >=25 or abs(prev_pts[0][0][0]-pts[0][0][0]) >=25:
            dummy = pts
            pts = prev_pts
            prev_pts = dummy
#             print( "SKIPPED!")
        else:
            prev_pts = pts
        dummy1 = latent1_pts
        dummy2 = latent2_pts
        newpts = (pts + latent3_pts + latent2_pts + latent1_pts) /4
        latent1_pts = pts
        latent2_pts = dummy1
        latent3_pts = dummy2
        pts = newpts
        
        

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, int([pts]), (0,255, 0))
#     print("points: ")
#     print(pts)
#     print("pts_vals: ")
#     print(pts[0,0])
#     print("pts_len: ")
#     print(pts[0,-1])
#     print(pts[0,300])
#     print("check")
    Minv = np.linalg.inv(Minv)
    pts_prev = pts

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    curv_info = "Avg curvature of the road is: " + str(float("{:.2f}".format(avg_curvature))) +"m"
    center = float("{:.2f}".format(center))
    if center < 0:
        center_info = "Vehicle is " + str(np.abs(center)) + "m left of center"
    else:
        center_info = "Vehicle is " + str(np.abs(center)) + "m right of center"
    cv2.putText(result,center_info,(30,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
    cv2.putText(result,curv_info,(30,30), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
#     plt.imshow(result)
    return result

In [20]:
def process_image(img):
    undist= undistort(img,mtx,dst)
    
    combined = sobel_combined(undist)
    combined = combined *255
    
    im, M = unwarp(combined)
    
    out_img, ploty, left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(im)    
    
    result = show_output_new(undist, im, M, ploty, left_fitx, right_fitx, left_fit, right_fit)
    return result

In [21]:
# import imageio
# imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML
isFirstFrame = True;
white_output = 'video1_output.mp4'
clip1 = VideoFileClip("video1.mp4")


white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
print("Start of writing!")
%time white_clip.write_videofile(white_output, audio=False)
print("Writing is completed!")

from IPython.display import HTML
white_output = 'video1_output.mp4'
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


ModuleNotFoundError: No module named 'moviepy'

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
isFirstFrame = True;
white_output = 'video2_output.mp4'
clip1 = VideoFileClip("video2.mp4")


white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
print("Start of writing!")
%time white_clip.write_videofile(white_output, audio=False)
print("Writing is completed!")

from IPython.display import HTML
white_output = 'video2_output.mp4'
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
isFirstFrame = True;
white_output = 'video3_output.mp4'
clip1 = VideoFileClip("video3.mp4")


white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
print("Start of writing!")
%time white_clip.write_videofile(white_output, audio=False)
print("Writing is completed!")

from IPython.display import HTML
white_output = 'video3_output.mp4'
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))